# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data.sqlite`.

In [ ]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [ ]:
# Your code here; create the connection
conn = sqlite3.Connection('data.sqlite')

## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [ ]:
# Your code here
q = """
    SELECT customerNumber, contactLastName, contactFirstName
    FROM customers 
    WHERE customerNumber IN (SELECT customerNumber FROM orders WHERE orderDate = '2003-01-31');
    """

In [ ]:
pd.read_sql(q, conn)

## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [ ]:
# Your code here
q = """
    SELECT p.productName, COUNT(o.orderNumber) AS numbersOrdered, SUM(o.quantityOrdered) AS totalUnitsSold
    FROM products AS p
    JOIN orders AS o
    ON p.productCode = o.productCode
    GROUP BY p.productName
    ORDER BY numberOrdered DESC;
    """

In [ ]:
pd.read_sql(q, conn)

## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [ ]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
q = """
    SELECT p.productName, COUNT(DISTINCT o.customerNumber) AS numPurchasers
    FROM products AS p
    JOIN orderdetails AS od
    ON p.productCode = od.productCode
    JOIN orders AS o
    ON od.orderNumber = o.orderNumber
    GROUP BY p.productName
    ORDER BY numPurchasers;
    """

In [ ]:
pd.read_sql(q, conn)

## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [ ]:
# Your code here
q = """
    SELECT DISTINCT e.employeeNumber, e.firstName, e.lastName, o.city, e.officeCode
    FROM employees AS e
    JOIN offices AS o
    ON e.officeCode = o.officeCode
    JOIN customers AS c
    ON c.salesRepEmployeeNumber = e.employeeNumber
    JOIN orders AS o
    ON c.customerNumber = o.customerNumber
    JOIN orderdetails AS od
    ON o.orderNumber = od.orderNumber
    WHERE productCode IN (
        SELECT productCode 
        FROM products AS p 
        JOIN orderdetails 
        ON p.productCode = od.productCode
        JOIN orders
        on o.orderNumber = od.orderNumber
        GROUP BY productCode
        HAVING COUNT(DISTINCT customerNumber) < 20
        );
    """

In [ ]:
pd.read_sql(q, conn)

## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [ ]:
# Your code here
q = """
    SELECT employeeNumber, firstName, lastName, COUNT(customerNumber) AS numCustomers
    FROM employees AS e
    JOIN customers As c
    ON e.employeeNumber = c.salesRepEmployeeNumber
    GROUP BY employeeNumber
    HAVING AVG(creditLimit) > 15000;
    """

In [ ]:
pd.read_sql(q, conn)

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!